In [1]:
import dxpy
import dxdata
import pandas as pd
import pyspark
import subprocess
from pyspark.sql import SQLContext
from pyspark import SparkConf, SparkContext

dxdata.__version__
connection = dxdata.connect()
conf = pyspark.SparkConf().set("spark.kryoserializer.buffer.max", "2000m")

/opt/conda/lib/python3.11/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self._context = ssl.SSLContext(ssl_version)


In [2]:
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [3]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']
dataset = dxdata.load_dataset(id=dispensed_dataset_id)
dataset.entities

[<Entity "participant">,
 <Entity "covid19_result_england">,
 <Entity "covid19_result_scotland">,
 <Entity "covid19_result_wales">,
 <Entity "gp_clinical">,
 <Entity "gp_scripts">,
 <Entity "gp_registrations">,
 <Entity "hesin">,
 <Entity "hesin_diag">,
 <Entity "hesin_oper">,
 <Entity "hesin_critical">,
 <Entity "hesin_maternity">,
 <Entity "hesin_delivery">,
 <Entity "hesin_psych">,
 <Entity "death">,
 <Entity "death_cause">,
 <Entity "olink_instance_0">,
 <Entity "olink_instance_2">,
 <Entity "olink_instance_3">]

In [4]:
participant = dataset['participant']

In [5]:
# Returns all field objects for a given UKB showcase field id
def fields_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given UKB showcase field id
def field_names_for_id(field_id):
    return [f.name for f in fields_for_id(field_id)]

In [6]:
# split the recommended fields into a list
merged_fields = pd.read_csv("merged_fields_3.tsv", sep="\t", header=0)
merge_field_list = merged_fields["field_id"].tolist()
# cut the recommended fields list for 20 fields each group
merge_field_list = [merge_field_list[i:i + 20] for i in range(0, len(merge_field_list), 20)]
for i, group in enumerate(merge_field_list):
    # if i <= 83:
    #     continue
    print(f"Retrieving fields for group {i + 1} with {len(group)} fields")
    # change the group astype to string
    group = [str(x) for x in group]
    get_ids = sum([field_names_for_id(field_id) for field_id in group], [])
    field_names = ['eid'] + get_ids
    df = participant.retrieve_fields(names=field_names, engine=connection)
    data = df.toPandas()
    display(data)
    # save the data to a file
    # if the file already exists, continue to the next iteration
    data.to_csv(f'./core_category/third_group_{i + 1}.tsv', sep='\t', index=False, header=True)
    # Upload the file using subprocess
    subprocess.run([
        "dx", "upload", f"./core_category/third_group_{i + 1}.tsv",
        "-p", "--path", "/Output/Traits/core_category/", "--brief"
    ], check=True)

Retrieving fields for group 1 with 20 fields


,eid,p3_i0,p3_i1,p3_i2,p3_i3,p4_i0,p4_i1,p4_i2,p4_i3,p5_i0,...,p630_i3,p5198_i0,p5198_i1,p5198_i2,p5198_i3,p6022_i0,p6022_i1,p10241_i0,p12148_i2,p12148_i3
0,1451876,390.0,657.0,NaN,NaN,501.0,542.0,NaN,NaN,28.0,...,NaN,NaN,545.0,NaN,NaN,NaN,1020.0,NaN,NaN,NaN
1,3046819,493.0,NaN,418.0,NaN,543.0,NaN,230.0,NaN,535.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,5015491,523.0,NaN,NaN,NaN,663.0,NaN,NaN,NaN,251.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1598419,395.0,NaN,531.0,NaN,467.0,NaN,199.0,NaN,275.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,3293483,508.0,NaN,NaN,NaN,252.0,NaN,NaN,NaN,299.0,...,NaN,560.0,NaN,NaN,NaN,445.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502123,2998069,936.0,NaN,NaN,NaN,739.0,NaN,NaN,NaN,458.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502124,3834454,332.0,NaN,266.0,NaN,576.0,NaN,861.0,NaN,301.0,...,NaN,362.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
502125,5758738,1127.0,NaN,NaN,NaN,324.0,NaN,NaN,NaN,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502126,5115563,635.0,NaN,NaN,NaN,462.0,NaN,NaN,NaN,256.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Retrieving fields for group 2 with 20 fields


,eid,p20118_i0,p21003_i0,p21003_i1,p21003_i2,p21003_i3,p21007,p21008,p21150,p21151,p21152,p21611_i2,p21611_i3,p21621_i2,p21621_i3,p21622_i2,p21622_i3,p21625_i2,p21625_i3
0,1451876,7.0,65,70.0,NaN,NaN,NaN,24.0,1.13448,-0.74872,0.73896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3046819,5.0,52,NaN,67.0,NaN,NaN,24.0,-0.19947,-0.03439,-1.08704,26.0,NaN,200.0,NaN,2707.0,NaN,1425.0,NaN
2,5015491,5.0,63,NaN,NaN,NaN,NaN,24.0,0.49424,-0.00576,0.39333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1598419,5.0,48,NaN,61.0,NaN,NaN,24.0,0.12488,-0.87493,-0.58550,47.0,NaN,238.0,NaN,2488.0,NaN,1876.0,NaN
4,3293483,7.0,63,NaN,NaN,NaN,NaN,24.0,1.34611,0.39447,0.96026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502123,2998069,5.0,54,NaN,NaN,NaN,NaN,24.0,0.08814,-0.71983,-0.92803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502124,3834454,5.0,41,NaN,48.0,NaN,NaN,24.0,0.81724,0.32466,0.09480,16.0,NaN,176.0,NaN,2270.0,NaN,1765.0,NaN
502125,5758738,5.0,64,NaN,NaN,NaN,NaN,24.0,1.00697,0.46150,0.04420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502126,5115563,5.0,45,NaN,NaN,NaN,NaN,24.0,0.09488,-1.23928,-1.98180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Retrieving fields for group 3 with 20 fields


...

,eid,p132564,p132565,p132566,p132567,p132568,p132569,p132570,p132571,p132572,...,p132574,p132575,p132576,p132577,p132578,p132579,p132580,p132581,p132582,p132583
0,1451876,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
1,3046819,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
2,5015491,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
3,1598419,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
4,3293483,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502123,2998069,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
502124,3834454,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
502125,5758738,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
502126,5115563,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN


Retrieving fields for group 149 with 20 fields


,eid,p132584,p132585,p132586,p132587,p132588,p132589,p132590,p132591,p132592,...,p132594,p132595,p132596,p132597,p132598,p132599,p132600,p132601,p132602,p132603
0,1451876,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
1,3046819,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
2,5015491,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
3,1598419,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
4,3293483,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502123,2998069,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
502124,3834454,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
502125,5758738,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN
502126,5115563,None,NaN,None,NaN,None,NaN,None,NaN,None,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN


Retrieving fields for group 150 with 2 fields


,eid,p132604,p132605
0,1451876,None,NaN
1,3046819,None,NaN
2,5015491,None,NaN
3,1598419,None,NaN
4,3293483,None,NaN
...,...,...,...
502123,2998069,None,NaN
502124,3834454,None,NaN
502125,5758738,None,NaN
502126,5115563,None,NaN
